In [ ]:
#@title Requirements
!pip install pandas
!pip install pandas_gbq

In [1]:
#@title Auth
from google.colab import auth
auth.authenticate_user()

In [2]:
#@title Dependencies
import pandas as pd
import pandas_gbq as pdgbq
import typing

In [16]:
#@title Data Retrieval

fmt_sql_list = lambda l : str(l).replace('[', '(').replace(']', ')')

def get_bq_view_dependencies(project : str, region : str, schemata : list) -> pd.DataFrame:  
  """Pulls a list of all view dependencies from the listed schemata."""
  
  sql = r"""
    with temp1_views as (
      select table_name as name, table_schema as `schema`, view_definition as definition
      from INFORMATION_SCHEMA.VIEWS
    ),
    temp2_tables as (
      select table_name as name, table_schema as `schema`, table_type as `type` 
      from INFORMATION_SCHEMA.TABLES
    ),
    temp3_dependencies as (
      select
        a.`schema`,
        a.name,
        array_agg(distinct concat(b.`schema`, '.', b.name)) as dependencies
      from 
        temp1_views a
        inner join temp2_tables b
          on regexp_contains(replace(a.definition, '`', ''), concat(b.`schema`, '.', b.name)  || '\\b')  --remove backticks
      where a.`schema` in INPUT_SCHEMATA
      group by 1, 2
    )
    select 
      dependency,
      concat(a.`schema`, '.', a.name) as view
    from temp3_dependencies a, unnest(a.dependencies) as dependency
    """.replace('INFORMATION_SCHEMA', region + '.' + 'INFORMATION_SCHEMA').replace('INPUT_SCHEMATA', fmt_sql_list(schemata))
  
  return pdgbq.read_gbq(sql,project_id = project)

def get_bq_tables(project : str, region : str, schemata : list) -> pd.DataFrame: # INFORMATION_SCHEMA.TABLES also includes views
  """Pulls a list of all tables and views from the listed schemata."""

  sql = """
      select table_name, table_schema, table_type
      from INFORMATION_SCHEMA.TABLES
      where table_schema in INPUT_SCHEMATA
  """.replace('INFORMATION_SCHEMA', region + '.' + 'INFORMATION_SCHEMA').replace('INPUT_SCHEMATA', fmt_sql_list(schemata))
  return pdgbq.read_gbq(sql,project_id = project)


In [18]:
#@title Mermaid Generation


def generate_mermaid(dependency_df: pd.DataFrame, table_df: pd.DataFrame, grouped_schemata : list) -> str:
  """Creates Mermaid code for dependencies, shaping table and view nodes differently and grouping specified schemata into subgraphs."""

  shape_node = lambda table_name, table_type : f'([{table_name}])' if table_type == 'VIEW' else f'({table_name})' # formats tables and views differently

  mermaid = 'graph TD;' 

  for dependency, view in zip(dependency_df.dependency, dependency_df.view):
    mermaid += f'\n\t{dependency} --> {view}'

  for schema in pd.unique(table_df.table_schema):
    schema_table_df = table_df[table_df.table_schema == schema]    
    
    if schema in grouped_schemata:
      mermaid += f'\n\tsubgraph {schema}'
      for table_name, table_type in zip(schema_table_df.table_name, schema_table_df.table_type):
        mermaid += f"\n\t\t{schema}.{table_name}{shape_node(table_name, table_type)}"
      mermaid += '\n\tend'
    else:
      for table_name, table_type in zip(schema_table_df.table_name, schema_table_df.table_type):
        mermaid += f"\n\t{schema}.{table_name}{shape_node(f'{schema}.{table_name}', table_type)}"    

  return mermaid

In [ ]:
print(
  generate_mermaid(
    get_bq_view_dependencies('project-id', 'region-id', ['schema1', 'schema2', 'schema3']),
    get_bq_tables('project-id', 'region-id', ['schema1', 'schema2', 'schema3']),
    ['schema2', 'schema3']
  )
)